In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [76]:
import tensorflow as tf
from tensorflow import keras as K
import matplotlib.pyplot as plt

In [100]:
train_data = pd.read_csv('//kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')


In [101]:
x_train = train_data.drop('label', axis=1).copy()
y_train = train_data['label'].copy()
x_test = test_data.copy()

In [102]:
x_train = x_train/255.0
x_test = x_test/255.0


In [103]:
x_train = x_train.values.reshape(-1, 28, 28, 1)
x_test = x_test.values.reshape(-1, 28, 28, 1)

In [106]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val=train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [104]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=120, kernel_size=(3,3), strides=(1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(filters=240, kernel_size=(3,3), strides=(1,1), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [107]:
history = model.fit(x=x_train, y=y_train, epochs=20, batch_size=64, validation_data=(x_val,y_val))

Epoch 1/20
591/591 [==============================] - 44s 75ms/step - loss: 0.0132 - accuracy: 0.9955 - val_loss: 0.0145 - val_accuracy: 0.9952
Epoch 2/20
591/591 [==============================] - 43s 74ms/step - loss: 0.0094 - accuracy: 0.9969 - val_loss: 0.0057 - val_accuracy: 0.9981
Epoch 3/20
591/591 [==============================] - 43s 73ms/step - loss: 0.0078 - accuracy: 0.9971 - val_loss: 0.0136 - val_accuracy: 0.9952
Epoch 4/20
591/591 [==============================] - 43s 74ms/step - loss: 0.0112 - accuracy: 0.9965 - val_loss: 0.0125 - val_accuracy: 0.9945
Epoch 5/20
591/591 [==============================] - 43s 73ms/step - loss: 0.0069 - accuracy: 0.9976 - val_loss: 0.0093 - val_accuracy: 0.9960
Epoch 6/20
591/591 [==============================] - 43s 72ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 0.0175 - val_accuracy: 0.9936
Epoch 7/20
591/591 [==============================] - 43s 73ms/step - loss: 0.0067 - accuracy: 0.9979 - val_loss: 0.0241 - val_accuracy:

In [109]:
final = model.predict(x_test)
final = np.argmax(final,axis = 1)
final = pd.Series(final, name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,len(x_test)+1),name = "ImageId"),final],axis = 1)

submission.to_csv("CNN_digit_recognizer.csv", index=False)